In [1]:
import numpy as np
import pandas as pd
import scipy
from scipy.io import loadmat
import pickle
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.stats import norm
import h5py
import os
import csv
from scipy.linalg import svd

In [2]:
# biobank_file = '../../Analysis/conf12.mat'
biobank_file = '../../../../workspace12eBETA_Andrei.mat'
mat = h5py.File(biobank_file, 'r')

In [3]:
mat.keys()

<KeysViewHDF5 ['#refs#', 'I', 'IDP_categories', 'IDP_category_names', 'IDP_names', 'IDP_reordering', 'IDPs', 'IDPs1', 'IDPs1_i', 'IDPs1_i_deconf', 'IDPs1_i_deconfnoage', 'IDPs2', 'IDPs2_i', 'IDPs2_i_deconf', 'IDPs2_i_deconfnoage', 'IDPs_i', 'IDPs_i_deconf', 'IDPs_i_deconfnoage', 'QC_IDP_names', 'QC_IDPs', 'QC_IDPs1', 'QC_IDPs2', 'WSNAME', 'age', 'age0', 'age1', 'age2', 'ans', 'birth_date', 'conf', 'conf1', 'conf12', 'conf2', 'conf_FST2', 'conf_NEWEDDY', 'conf_SCALING_SWI', 'conf_SCALING_T1', 'conf_SCALING_T2', 'conf_SCALING_dMRI', 'conf_SCALING_rfMRI', 'conf_SCALING_tfMRI', 'conf_TE_rfMRI', 'conf_TE_tfMRI', 'conf_TablePos_COG_Table', 'conf_TablePos_COG_X', 'conf_TablePos_COG_Y', 'conf_TablePos_COG_Z', 'conf_YTRANS', 'conf_groups_names', 'conf_names', 'grot', 'head_size_scaling', 'ind_conf_groups', 'ind_site', 'nIDPsDIR', 'num_conf_groups', 'omp_threads', 'omp_threads_str', 'repeated_IDs', 'rfMRI_ICA_feature_weights', 'scan_date', 'scan_date1', 'scan_date2', 'scan_number', 'scan_pairs',

In [4]:
np.transpose(mat['conf12'],(1,0)).shape

(42897, 613)

In [5]:
subject_IDs_unique = np.transpose(mat['subject_IDs_unique'], (1,0)).flatten().astype(int)
conf12 = np.transpose(mat['conf12'],(1,0))

In [6]:
for key in list(mat.keys()):
    print(key, mat[key])

#refs# <HDF5 group "/#refs#" (39725 members)>
I <HDF5 dataset "I": shape (1, 1), type "<f8">
IDP_categories <HDF5 dataset "IDP_categories": shape (1, 3921), type "<f8">
IDP_category_names <HDF5 dataset "IDP_category_names": shape (16, 1), type "|O">
IDP_names <HDF5 dataset "IDP_names": shape (1, 3921), type "|O">
IDP_reordering <HDF5 dataset "IDP_reordering": shape (1, 3921), type "<f8">
IDPs <HDF5 dataset "IDPs": shape (3921, 46743), type "<f8">
IDPs1 <HDF5 dataset "IDPs1": shape (3921, 42897), type "<f8">
IDPs1_i <HDF5 dataset "IDPs1_i": shape (3921, 42897), type "<f8">
IDPs1_i_deconf <HDF5 dataset "IDPs1_i_deconf": shape (3921, 42897), type "<f8">
IDPs1_i_deconfnoage <HDF5 dataset "IDPs1_i_deconfnoage": shape (3921, 42897), type "<f8">
IDPs2 <HDF5 dataset "IDPs2": shape (3921, 42897), type "<f8">
IDPs2_i <HDF5 dataset "IDPs2_i": shape (3921, 42897), type "<f8">
IDPs2_i_deconf <HDF5 dataset "IDPs2_i_deconf": shape (3921, 42897), type "<f8">
IDPs2_i_deconfnoage <HDF5 dataset "IDPs2_i_

# 1. Male & Female Test Datasets for IDPs and nIDPs

In [7]:
male_test = np.loadtxt('male_test.txt').astype(int)
female_test = np.loadtxt('female_test.txt').astype(int)

male_validation = np.loadtxt('male_validation.txt').astype(int)
female_validation = np.loadtxt('female_validation.txt').astype(int)

female_testA = np.loadtxt('female_testA.txt').astype(int)
female_testB = np.loadtxt('female_testB.txt').astype(int)

male_testA = np.loadtxt('male_testA.txt').astype(int)
male_testB = np.loadtxt('male_testB.txt').astype(int)

In [8]:
counter = 0
for subject in subject_IDs_unique:
    subject = int('2' + str(subject))
    if subject in male_test:
        counter += 1
        
print(len(male_test), counter)

4364 4364


In [9]:
counter = 0
for subject in male_test:
    subj = subject - 20000000
    if subj not in subject_IDs_unique:
        print('Exlude the following subject:', subject)

In [10]:
counter = 0
for subject in subject_IDs_unique:
    subject = int('2' + str(subject))
    if subject in male_validation:
        counter += 1
        
print(len(male_validation), counter)

1091 1091


In [11]:
counter = 0
for subject in subject_IDs_unique:
    subject = int('2' + str(subject))
    if subject in female_test:
        counter += 1
        
print(len(female_test), counter)

5020 5020


In [12]:
counter = 0
for subject in subject_IDs_unique:
    subject = int('2' + str(subject))
    if subject in female_validation:
        counter += 1
        
print(len(female_validation), counter)

1255 1255


In [13]:
subjects_male = []
conf12_male = []

for subject in male_test:
    subj = subject - 20000000
    if subj in subject_IDs_unique:
        idx = np.where(subject_IDs_unique == subj)
        subjects_male.append(subject)
        conf12_male.append(conf12[idx].flatten())
        
subjects_male = np.array(subjects_male)
conf12_male = np.array(conf12_male)

conf12_male_processed = conf12_male - conf12_male.mean(axis=0)
conf12_male_processed_rank = np.linalg.matrix_rank(conf12_male_processed)
if np.sum(np.isnan(conf12_male_processed)) == 0:
    conf12_male_processed,_,_ = svd(conf12_male_processed)
    conf12_male_processed = conf12_male_processed[:, :conf12_male_processed_rank]


if os.path.exists('male_test_conf12.h5'):
    os.remove('male_test_conf12.h5')
h5f = h5py.File('male_test_conf12.h5', 'w')
h5f.create_dataset('subjects', data=subjects_male)
h5f.create_dataset('conf12', data=conf12_male)
h5f.create_dataset('conf12proc', data=conf12_male_processed)
h5f.close()

In [14]:
subjects_female = []
conf12_female = []

for subject in female_testA:
    subj = subject - 20000000
    if subj in subject_IDs_unique:
        idx = np.where(subject_IDs_unique == subj)
        subjects_female.append(subject)
        conf12_female.append(conf12[idx].flatten())
        
subjects_female = np.array(subjects_female)
conf12_female = np.array(conf12_female)

conf12_female_processed = conf12_female - conf12_female.mean(axis=0)
conf12_female_processed_rank = np.linalg.matrix_rank(conf12_female_processed)
if np.sum(np.isnan(conf12_female_processed)) == 0:
    conf12_female_processed,_,_ = svd(conf12_female_processed)
    conf12_female_processed = conf12_female_processed[:, :conf12_female_processed_rank]


if os.path.exists('female_testA_conf12.h5'):
    os.remove('female_testA_conf12.h5')
h5f = h5py.File('female_testA_conf12.h5', 'w')
h5f.create_dataset('subjects', data=subjects_female)
h5f.create_dataset('conf12', data=conf12_female)
h5f.create_dataset('conf12proc', data=conf12_female_processed)
h5f.close()

In [15]:
subjects_male = []
conf12_male = []

for subject in male_testA:
    subj = subject - 20000000
    if subj in subject_IDs_unique:
        idx = np.where(subject_IDs_unique == subj)
        subjects_male.append(subject)
        conf12_male.append(conf12[idx].flatten())
        
subjects_male = np.array(subjects_male)
conf12_male = np.array(conf12_male)

conf12_male_processed = conf12_male - conf12_male.mean(axis=0)
conf12_male_processed_rank = np.linalg.matrix_rank(conf12_male_processed)
if np.sum(np.isnan(conf12_male_processed)) == 0:
    conf12_male_processed,_,_ = svd(conf12_male_processed)
    conf12_male_processed = conf12_male_processed[:, :conf12_male_processed_rank]


if os.path.exists('male_testA_conf12.h5'):
    os.remove('male_testA_conf12.h5')
h5f = h5py.File('male_testA_conf12.h5', 'w')
h5f.create_dataset('subjects', data=subjects_male)
h5f.create_dataset('conf12', data=conf12_male)
h5f.create_dataset('conf12proc', data=conf12_male_processed)
h5f.close()

In [14]:
subjects_male = []
conf12_male = []

for subject in male_testB:
    subj = subject - 20000000
    if subj in subject_IDs_unique:
        idx = np.where(subject_IDs_unique == subj)
        subjects_male.append(subject)
        conf12_male.append(conf12[idx].flatten())
        
subjects_male = np.array(subjects_male)
conf12_male = np.array(conf12_male)

conf12_male_processed = conf12_male - conf12_male.mean(axis=0)
conf12_male_processed_rank = np.linalg.matrix_rank(conf12_male_processed)
if np.sum(np.isnan(conf12_male_processed)) == 0:
    conf12_male_processed,_,_ = svd(conf12_male_processed)
    conf12_male_processed = conf12_male_processed[:, :conf12_male_processed_rank]


if os.path.exists('male_testB_conf12.h5'):
    os.remove('male_testB_conf12.h5')
h5f = h5py.File('male_testB_conf12.h5', 'w')
h5f.create_dataset('subjects', data=subjects_male)
h5f.create_dataset('conf12', data=conf12_male)
h5f.create_dataset('conf12proc', data=conf12_male_processed)
h5f.close()

In [18]:
subjects_female = []
conf12_female = []

for subject in female_testB:
    subj = subject - 20000000
    if subj in subject_IDs_unique:
        idx = np.where(subject_IDs_unique == subj)
        subjects_female.append(subject)
        conf12_female.append(conf12[idx].flatten())
        
subjects_female = np.array(subjects_female)
conf12_female = np.array(conf12_female)

conf12_female_processed = conf12_female - conf12_female.mean(axis=0)
conf12_female_processed_rank = np.linalg.matrix_rank(conf12_female_processed)
if np.sum(np.isnan(conf12_female_processed)) == 0:
    conf12_female_processed,_,_ = svd(conf12_female_processed)
    conf12_female_processed = conf12_female_processed[:, :conf12_female_processed_rank]


if os.path.exists('female_testB_conf12.h5'):
    os.remove('female_testB_conf12.h5')
h5f = h5py.File('female_testB_conf12.h5', 'w')
h5f.create_dataset('subjects', data=subjects_female)
h5f.create_dataset('conf12', data=conf12_female)
h5f.create_dataset('conf12proc', data=conf12_female_processed)
h5f.close()

In [19]:
subjects_male = []
conf12_male = []

for subject in male_validation:
    subj = subject - 20000000
    if subj in subject_IDs_unique:
        idx = np.where(subject_IDs_unique == subj)
        subjects_male.append(subject)
        conf12_male.append(conf12[idx].flatten())
        
subjects_male = np.array(subjects_male)
conf12_male = np.array(conf12_male)

conf12_male_processed = conf12_male - conf12_male.mean(axis=0)
conf12_male_processed_rank = np.linalg.matrix_rank(conf12_male_processed)
if np.sum(np.isnan(conf12_male_processed)) == 0:
    conf12_male_processed,_,_ = svd(conf12_male_processed)
    conf12_male_processed = conf12_male_processed[:, :conf12_male_processed_rank]


if os.path.exists('male_validation_conf12.h5'):
    os.remove('male_validation_conf12.h5')
h5f = h5py.File('male_validation_conf12.h5', 'w')
h5f.create_dataset('subjects', data=subjects_male)
h5f.create_dataset('conf12', data=conf12_male)
h5f.create_dataset('conf12proc', data=conf12_male_processed)
h5f.close()

In [20]:
subjects_female = []
conf12_female = []

for subject in female_test:
    subj = subject - 20000000
    if subj in subject_IDs_unique:
        idx = np.where(subject_IDs_unique == subj)
        subjects_female.append(subject)
        conf12_female.append(conf12[idx].flatten())
        
subjects_female = np.array(subjects_female)
conf12_female = np.array(conf12_female)

conf12_female_processed = conf12_female - conf12_female.mean(axis=0)
conf12_female_processed_rank = np.linalg.matrix_rank(conf12_female_processed)
if np.sum(np.isnan(conf12_female_processed)) == 0:
    conf12_female_processed,_,_ = svd(conf12_female_processed)
    conf12_female_processed = conf12_female_processed[:, :conf12_female_processed_rank]

if os.path.exists('female_test_conf12.h5'):
    os.remove('female_test_conf12.h5')
h5f = h5py.File('female_test_conf12.h5', 'w')
h5f.create_dataset('subjects', data=subjects_female)
h5f.create_dataset('conf12', data=conf12_female)
h5f.create_dataset('conf12proc', data=conf12_female_processed)
h5f.close()

In [21]:
subjects_female = []
conf12_female = []

for subject in female_validation:
    subj = subject - 20000000
    if subj in subject_IDs_unique:
        idx = np.where(subject_IDs_unique == subj)
        subjects_female.append(subject)
        conf12_female.append(conf12[idx].flatten())
        
subjects_female = np.array(subjects_female)
conf12_female = np.array(conf12_female)

conf12_female_processed = conf12_female - conf12_female.mean(axis=0)
conf12_female_processed_rank = np.linalg.matrix_rank(conf12_female_processed)
if np.sum(np.isnan(conf12_female_processed)) == 0:
    conf12_female_processed,_,_ = svd(conf12_female_processed)
    conf12_female_processed = conf12_female_processed[:, :conf12_female_processed_rank]

if os.path.exists('female_validation_conf12.h5'):
    os.remove('female_validation_conf12.h5')
h5f = h5py.File('female_validation_conf12.h5', 'w')
h5f.create_dataset('subjects', data=subjects_female)
h5f.create_dataset('conf12', data=conf12_female)
h5f.create_dataset('conf12proc', data=conf12_female_processed)
h5f.close()

In [22]:
print(conf12_female.shape)

(1255, 613)


In [23]:
test = h5py.File('male_testB_conf12.h5', 'r')
test.keys()

<KeysViewHDF5 ['conf12', 'conf12proc', 'subjects']>

In [24]:
for key in list(test.keys()):
    print(key, test[key].shape)

conf12 (2182, 613)
conf12proc (2182, 546)
subjects (2182,)


# 2. Male and Female Scan-Rescan Test Datasets for nIDPs and IDPs

In [54]:
test.keys()

<KeysViewHDF5 ['conf12', 'subjects']>

In [55]:
test['subjects'][()][0:5]

array([24806531, 24237795, 24748274, 25463415, 25313661])

In [56]:
test['conf12']

<HDF5 dataset "conf12": shape (5021, 613), type "<f8">